# Decision tree that will help decide which stocks are a valid buy


## Getting the data


In [6]:
import sqlite3 as sql
import pandas as pd
import yfinance as yf

In [7]:


def get_cash_flow_increase(cash_flow):
      i=3
      old_cf = 0
      while i >-1:
            year = cash_flow.columns[i]

            cf = cash_flow[year].loc['Free Cash Flow']

            if float(cf) > old_cf:
                  old_cf = cf
            else:
                  return False
            
            i-=1
            
      return True



In [11]:




#top 50 stocks by market cap
stock_tickers = ["MSFT","CHGG", "AAPL", "NVDA", "GOOG", "GOOGL", "AMZN", "META",
                 "LLY", "AVGO", "TSLA", "V", "WMT", "XOM", "UNH", "MA", 
                 "ORCL", "COST", "ADBE", "NFLX", "CVX", "AMD", "KO", "ABBV", 
                 "MRK", "PEP", "TMO", "ACN", "CRM", "ABT", "LIN", "MCD", 
                 "MDT", "AMGN", "PYPL", "INTC", "HON", "RTX", "TXN", "QCOM", 
                 "UPS", "NKE", "PM", "SBUX", "IBM", "BMY", "PFE", "CAT", "SPGI", "DHR"
]


stock_info_dfs = []

for stock in stock_tickers:

    ticker = yf.Ticker(stock)

    balance_sheet = ticker.balance_sheet

    income_statement = ticker.income_stmt

    cash_flow = ticker.cash_flow

    # categories = cash_flow.index.tolist()

    #displays all category names in the balance shet, income statement or cash flow report depending on which is entered
    # for category in categories:
    #     print(category)

    #data that will be used to determine the validity of a stock buy

    #create the database

    
    
    most_recent_year_b = balance_sheet.columns[0]
    most_recent_year_istmt = income_statement.columns[0]
    
    current_assets = balance_sheet[most_recent_year_b].loc['Current Assets']
    current_liabilities = balance_sheet[most_recent_year_b].loc['Current Liabilities']
    operating_income = income_statement[most_recent_year_istmt].loc['Operating Income']
    total_revenue = income_statement[most_recent_year_istmt].loc['Total Revenue']
    free_cash_flow_4 = cash_flow.loc['Free Cash Flow']

    
    data = {
        'Total Current Assets': current_assets,
        'Total Current Liabilities': current_liabilities,
        'Operating Income': operating_income,
        'Total Revenue': total_revenue,
        'Free Cash Flow over last 4 years': free_cash_flow_4
    }

   

    
    if float(current_assets/current_liabilities) > 1 and float(operating_income/total_revenue) > .15 and get_cash_flow_increase(cash_flow):
        data.update({'Buy': 1})
    else:
        data.update({'Buy': 0})
        
        



    data = pd.DataFrame(data)

    stock_info_dfs.append(data)


stock_dict = dict(zip(stock_tickers, stock_info_dfs))












In [12]:
#create SQLite Database

conn = sql.connect('stock_criteria.db')

for ticker, df in stock_dict.items():
    df.to_sql(ticker, conn, if_exists='replace', index = False)

conn.close()



## Node Class

In [10]:
class Node:
    def __init__(self, feature_index = None, threshold = None, left = None, right = None, info_gain = None, value = None):
        
        #for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain

        #for leaf node
        self.value = value
